In [ ]:
import pandas as pd
import geopandas as gpd
import folium
#ok

In [ ]:
distritos = gpd.read_file("./SIRGAS_SHP_distrito/SIRGAS_SHP_distrito.shp")
populacao = pd.read_csv("./Agregados_por_CEP_CNEFE_Censo_2022/Agregados_por_CEP_CNEFE_Censo_2022.csv", sep = ";" )
estacoes = pd.read_parquet("estacoes_reestruturadas.parquet")
declividade = gpd.read_file("./declividade/sad6996_declividade.shp")

In [ ]:
distritos

In [ ]:
populacao.head()

In [ ]:
estacoes.head()

In [ ]:
declividade.head()

In [ ]:
declividade.shape

In [ ]:
declividade.isnull().sum()

In [ ]:
declividade['CLASSE'].value_counts()

## Transformação de classes 
- 0 a 05% = 1
- 05 a 25% = 2
- 25 a 60% = 3
- maior que 60% = 4

In [ ]:
# Dicionário
substituicoes = {
    '0 a 05%': 1,
    '05 a 25%': 2,
    '25 a 60%': 3,
    '> 60%': 4
}

# Substituindo valores
declividade['CLASSE'] = declividade['CLASSE'].replace(substituicoes).astype(int)

In [ ]:
latitude = -23.550520
longitude = -46.633308
mapa_sp = folium.Map(location = [latitude, longitude], zoom_start = 12)

In [ ]:
# mapa_sp

In [ ]:
declividade = declividade.set_crs("EPSG:29183")  # Define o CRS, se ainda não estiver definido
declividade = declividade.to_crs("EPSG:4326")   # Converte para WGS84 (latitude/longitude)

In [ ]:

def get_color(valor):
    if valor == 1:
        return "yellow"
    elif valor == 2:
        return "orange"
    elif valor == 3:
        return "red"
    elif valor == 4:
        return "darkred"
    else:
        return "black" # error


In [ ]:
## Adicionando geometrias ao mapa coloridas
for _, row in declividade.iterrows():
        cor = get_color(row['CLASSE'])
    
        folium.GeoJson(
            row['geometry'].__geo_interface__, 
            name = row.get('ID', 'Sem id'),
            tooltip = row.get('ID', 'Sem id'),
            style_function = lambda x, color = cor:{'fillColor' : color , 'color' : color, 'weight' : 1}
        ).add_to(mapa_sp)

In [ ]:
mapa_sp

In [ ]:
declividade